In [9]:
import pandas as pd
import numpy as np
from plotnine import *

%matplotlib inline

In [25]:
data = pd.read_csv("../../data/processed/expensas_full_processed_test_lag_3.csv",
                  dtype = {
                                "expensa_id": "object",
                                "unidad_id": "object",
                                "consorcio_id": "object",
                                "expensa_mes":"object",
                            },
                  parse_dates = ["expensa_fecha", "expensa_mes_pago", "expensa_mes_pago_anterior"])

In [26]:
data.shape

(21985, 21)

In [27]:
data.dtypes

expensa_id                                      object
consorcio_id                                    object
consorcio_nombre                                object
consorcio_cantidad_expensas                    float64
consorcio_prop_impagos                         float64
consorcio_prop_efectivo                        float64
unidad_id                                       object
unidad_denominacion                             object
unidad_tipo                                     object
unidad_prop_impagos                            float64
expensa_proporcion                             float64
expensa_interes_primer_vencimiento             float64
expensa_ineteres_segundo_vencimiento           float64
expensa_mes                                     object
expensa_fecha                           datetime64[ns]
expensa_mes_pago                        datetime64[ns]
expensa_mes_pago_anterior               datetime64[ns]
pago_metodo_lag_1                               object
pago_metod

In [28]:
data = data.dropna()

In [11]:
data.columns

Index(['expensa_id', 'consorcio_id', 'consorcio_nombre',
       'consorcio_cantidad_expensas', 'consorcio_prop_impagos',
       'consorcio_prop_efectivo', 'unidad_id', 'unidad_denominacion',
       'unidad_tipo', 'unidad_prop_impagos', 'expensa_proporcion',
       'expensa_interes_primer_vencimiento',
       'expensa_ineteres_segundo_vencimiento', 'expensa_mes', 'expensa_fecha',
       'expensa_mes_pago', 'expensa_mes_pago_anterior', 'pago_metodo_lag_1',
       'pago_metodo_lag_2', 'pago_metodo_lag_3', 'target'],
      dtype='object')

In [37]:
X_columns  = [ 'consorcio_cantidad_expensas', 'consorcio_prop_impagos',
               'consorcio_prop_efectivo',
               'unidad_tipo', 'unidad_prop_impagos', 
               'expensa_proporcion',
               'expensa_interes_primer_vencimiento',
               'expensa_ineteres_segundo_vencimiento', 'expensa_mes',
               'pago_metodo_lag_1', 'pago_metodo_lag_2', 'pago_metodo_lag_3']

## Import Model and Scaler

In [52]:
import pickle
with open('../../models/logisticRegression2.pkl', 'rb') as file:
    logReg = pickle.load(file)
with open('../../models/minmaxScaler2.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('../../models/columns_list2.pkl', 'rb') as file:
    column_list = pickle.load(file)

In [53]:
X = data[X_columns]

In [54]:
X = pd.get_dummies(X)

In [55]:
for column in [column for column in column_list if column not in X.columns]:
    X[column] = 0

In [56]:
X.shape

(17418, 42)

In [57]:
X.consorcio_cantidad_expensas = scaler.transform(X[["consorcio_cantidad_expensas"]])

In [58]:
Y = data.target

In [59]:
X.isna().sum()

consorcio_cantidad_expensas             0
consorcio_prop_impagos                  0
consorcio_prop_efectivo                 0
unidad_prop_impagos                     0
expensa_proporcion                      0
expensa_interes_primer_vencimiento      0
expensa_ineteres_segundo_vencimiento    0
unidad_tipo_Baulera                     0
unidad_tipo_Casa                        0
unidad_tipo_Cochera                     0
unidad_tipo_Departamento                0
unidad_tipo_Duplex                      0
unidad_tipo_Local                       0
unidad_tipo_Lote                        0
unidad_tipo_Oficina                     0
expensa_mes_01                          0
expensa_mes_02                          0
expensa_mes_12                          0
pago_metodo_lag_1_Efec-Cheque           0
pago_metodo_lag_1_EntePago              0
pago_metodo_lag_1_Impago                0
pago_metodo_lag_1_Internet              0
pago_metodo_lag_1_Otro                  0
pago_metodo_lag_2_Efec-Cheque     

In [60]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
preds = logReg.predict(X)
print("accuracy:", accuracy_score(Y, preds))
print("precision:", precision_score(Y, preds))
print("recall:", recall_score(Y, preds))
print("f1:", f1_score(Y, preds))

accuracy: 0.7527270639568263
precision: 0.4771986970684039
recall: 0.06847394250993223
f1: 0.11976292662987942


In [61]:
with open('../../models/xgb_lag3.pkl', 'rb') as file:
    xgb = pickle.load(file)

In [63]:
preds = xgb.predict(X.values)
print("accuracy:", accuracy_score(Y, preds))
print("precision:", precision_score(Y, preds))
print("recall:", recall_score(Y, preds))
print("f1:", f1_score(Y, preds))

accuracy: 0.7701802732805144
precision: 0.709726443768997
recall: 0.10913764898340733
f1: 0.1891837148065627


## Linear Regression Pagos historicos

In [38]:
import pickle
with open('../../models/logisticRegressionFeatureSelection.pkl', 'rb') as file:
    logRegHp = pickle.load(file)

In [39]:
train_columns = ['unidad_prop_impagos',
 'expensa_proporcion',
 'expensa_interes_primer_vencimiento',
 'expensa_ineteres_segundo_vencimiento',
 'unidad_tipo_Casa',
 'unidad_tipo_Departamento',
 'unidad_tipo_Duplex',
 'unidad_tipo_Lote',
 'unidad_tipo_Oficina',
 'expensa_mes_03',
 'expensa_mes_05',
 'expensa_mes_07',
 'expensa_mes_08',
 'expensa_mes_11',
 'expensa_mes_12',
 'pago_metodo_lag_1_Impago',
 'pago_metodo_lag_1_Internet',
 'pago_metodo_lag_1_Otro',
 'pago_metodo_lag_2_EntePago',
 'pago_metodo_lag_2_Internet',
 'pago_metodo_lag_2_Otro',
 'pago_metodo_lag_3_EntePago',
 'pago_metodo_lag_3_Internet',
 'pago_metodo_lag_3_Otro']

In [40]:
data_hp = pd.get_dummies(data[X_columns])

In [42]:
for column in [column for column in train_columns if column not in data_hp.columns]:
    data_hp[column]=0

In [44]:
data_hp.shape

(17418, 38)

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
Y = data.target
preds = logRegHp.predict(data_hp[train_columns])
print("accuracy:", accuracy_score(Y, preds))
print("precision:", precision_score(Y, preds))
print("recall:", recall_score(Y, preds))
print("f1:", f1_score(Y, preds))

accuracy: 0.8616947984843265
precision: 0.7617581187010078
recall: 0.6358962374386539
f1: 0.6931601069927398
